In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
   .master("local") \
   .appName("Predicting Fire Dept Calls") \
   .config("spark.executor.memory", "6gb") \
   .getOrCreate()

In [ ]:
df = spark.read.format('com.databricks.spark.csv')\
                    .options(header='true', inferschema='true')\
                    .load('Fire_Department_Calls_for_Service.csv')


In [ ]:
df.show(2)

In [ ]:
df.select('Call Type Group').distinct().show()

In [ ]:
df.groupBy('Call Type Group').count().show()

In [ ]:
df2 = df.groupBy('Call Type Group').count()

In [ ]:
graphDF = df2.toPandas()
graphDF = graphDF.sort_values('count', ascending=False)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
graphDF.plot(x='Call Type Group', y = 'count', kind='bar')
plt.title('Call Type Group by Count')
plt.show()

In [ ]:
df.groupBy('Call Type').count().orderBy('count', ascending=False).show(100)

In [ ]:
from pyspark.sql import functions as F
fireIndicator = df.select(df["Call Type"],F.when(df["Call Type"].like("%Fire%"),1)\
                          .otherwise(0).alias('Fire Indicator'))
fireIndicator.show()

In [ ]:
fireIndicator.groupBy('Fire Indicator').count().show()

In [ ]:
df = df.withColumn("fireIndicator", F.when(df["Call Type"].like("%Fire%"),1).otherwise(0))

In [ ]:
df.printSchema()

In [ ]:
df.select('Call Type', 'fireIndicator').show(20)

In [ ]:
df = df.select('fireIndicator', 
               'Zipcode of Incident',
               'Battalion',
               'Station Area',
               'Box', 
               'Number of Alarms',
               'Unit sequence in call dispatch',
               'Neighborhooods - Analysis Boundaries',
               'Fire Prevention District',
               'Supervisor District')
df.show(5)

In [ ]:
print('Total Rows')
df.count()

In [ ]:
print('Rows without Null values')
df.dropna().count()

In [ ]:
print('Row with Null Values')
df.count()-df.dropna().count()

In [ ]:
df = df.dropna()

In [ ]:
df.groupBy('fireIndicator').count().orderBy('count', ascending = False).show()

In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
column_names = df.columns[1:]
column_names

In [ ]:
categoricalColumns = column_names
indexers = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"_Index")
    indexers += [stringIndexer]

In [ ]:
models = []
for model in indexers:
    indexer_model = model.fit(df)
    models+=[indexer_model]
    
for i in models:
    df = i.transform(df)

In [ ]:
df.columns

In [ ]:
df.select('Neighborhooods - Analysis Boundaries', 'Neighborhooods - Analysis Boundaries_Index').show()

In [ ]:
df = df.select(
          'fireIndicator',
          'Zipcode of Incident_Index',
          'Battalion_Index',
          'Station Area_Index',
          'Box_Index',
          'Number of Alarms_Index',
          'Unit sequence in call dispatch_Index',
          'Neighborhooods - Analysis Boundaries_Index',
          'Fire Prevention District_Index',
          'Supervisor District_Index')

In [ ]:
df.printSchema()

In [ ]:
df.show(5)

In [ ]:
features = df.columns[1:]

In [ ]:
from pyspark.ml.feature import VectorAssembler

feature_vectors = VectorAssembler(
        inputCols = features,
        outputCol = "features")

In [ ]:
df = feature_vectors.transform(df)

In [ ]:
df.columns

In [ ]:
df = df.drop( 'Zipcode of Incident_Index',
              'Battalion_Index',
              'Station Area_Index',
              'Box_Index',
              'Number of Alarms_Index',
              'Unit sequence in call dispatch_Index',
              'Neighborhooods - Analysis Boundaries_Index',
              'Fire Prevention District_Index',
              'Supervisor District_Index')

In [ ]:
df = df.withColumnRenamed('fireIndicator', 'label')

In [ ]:
df.show()

In [ ]:
(trainDF, testDF) = df.randomSplit([0.75, 0.25], seed = 12345)

In [ ]:
print(trainDF.count())
print(testDF.count())

In [ ]:
from pyspark.ml.classification import LogisticRegression
logreg = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)
LogisticRegressionModel = logreg.fit(trainDF)

In [ ]:
df_predicted = LogisticRegressionModel.transform(testDF)

In [ ]:
df_predicted.printSchema()

In [ ]:
df_predicted.show(5)

In [ ]:
df_predicted.crosstab('label', 'prediction').show()

In [ ]:
from sklearn import metrics

In [ ]:
actual = df_predicted.select('label').toPandas()

In [ ]:
predicted = df_predicted.select('prediction').toPandas()

In [ ]:
metrics.accuracy_score(actual, predicted)

In [ ]:
df_predicted.groupBy('label').count().show()

In [ ]:
df_predicted.describe('label').show()